### **BPIC 2012 - No milestone**


In [11]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from data.processor import *
from data.args import *

from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

### Define the experiment parameters

In [2]:
milestone = 'All' #'A_PREACCEPTED' # 'W_Nabellen offertes', 'All'
subset = "O"
experiment =  'OHE'#'Standard'#'OHE', 'No_loops'
n_size = 5
max_size = 1000 #  3, 5, 10, 15, 20, 30, 50, 95
min_size = 0  # 0, 3, 5, 10, 15, 20, 30, 50

### Setting up the key parameters


In [3]:
# For local environment only
MY_WORKSPACE_DIR = os.path.join(os.getcwd(),'BPIC12')
MILESTONE_DIR = os.path.join(os.path.join(MY_WORKSPACE_DIR, milestone),experiment)

In [4]:
args = get_parameters('bpic12', MILESTONE_DIR, MY_WORKSPACE_DIR, milestone, experiment,n_size)
args['file_name_O_all'] = os.path.join(os.path.join(MY_WORKSPACE_DIR, "Translated_dataset"), "bpic12_translated_completed_O_all.csv" )

### Data Preprocessing

In [5]:
log_df = pd.read_csv(args['file_name_O_all'])
log_df = log_df.reset_index(drop=True)

log_df.head()

prefix_id  caseid        task             role                end_timestamp  \
0  173796_1  173796  O_SELECTED  role_     11001  2011-10-03 19:15:53.5370000   
1  173835_1  173835  O_SELECTED  role_     11180  2011-10-03 18:35:59.2790000   
2  173871_4  173871  O_SELECTED  role_     10909  2011-10-05 03:40:46.5260000   
3  173871_4  173871   O_CREATED  role_     10909  2011-10-05 03:40:47.7410000   
4  173871_4  173871      O_SENT  role_     10909  2011-10-05 03:40:47.8250000   

                   trace_start  timelapsed next_activity    milestone  \
0  2011-10-03 19:15:53.5370000           0     O_CREATED   O_SELECTED   
1  2011-10-03 18:35:59.2790000           0     O_CREATED   O_SELECTED   
2  2011-10-05 03:40:46.5260000           0    O_DECLINED  O_SENT_BACK   
3  2011-10-05 03:40:46.5260000           0    O_DECLINED  O_SENT_BACK   
4  2011-10-05 03:40:46.5260000           0    O_DECLINED  O_SENT_BACK   

   milestone_id  task_index  prefix_length  
0             1           1              1  
1             1           1              1  
2             4           1              4  
3             4           2              4  
4             4           3              4

Milestone Extraction

We expect to filter the prefixes by the milestone, and train the model for each milestone separately to observe if there will be an improvement in the model explainability as well as performance


In [6]:
if milestone != 'All':
  log_df = log_df[log_df['milestone'] == milestone]
else:
  log_df = log_df[(log_df['prefix_length'] > min_size) & (log_df['prefix_length'] <= max_size) ]

In [7]:
# Replacing empty cells of categorical columns with 'none'
log_df = log_df.fillna('none')

Index creating for data encoding
Function : `create_index(log_df, column)`, Author: Manuel Camargo, Edited by: Bemali Wickramanayake

In [8]:
# Index creation for activity
ac_index = create_index(log_df, 'task')
index_ac = {v: k for k, v in ac_index.items()}

# Index creation for unit
rl_index = create_index(log_df, 'role')
index_rl = {v: k for k, v in rl_index.items()}

# Index creation for next activity
ne_index = create_index(log_df, 'next_activity')
index_ne = {v: k for k, v in ne_index.items()}

#mapping the dictionary values as columns in the dataframe
log_df['ac_index'] = log_df['task'].map(ac_index)
log_df['rl_index'] = log_df['role'].map(rl_index)
log_df['ne_index'] = log_df['next_activity'].map(ne_index)

log_df.head()

prefix_id  caseid        task             role                end_timestamp  \
0  173796_1  173796  O_SELECTED  role_     11001  2011-10-03 19:15:53.5370000   
1  173835_1  173835  O_SELECTED  role_     11180  2011-10-03 18:35:59.2790000   
2  173871_4  173871  O_SELECTED  role_     10909  2011-10-05 03:40:46.5260000   
3  173871_4  173871   O_CREATED  role_     10909  2011-10-05 03:40:47.7410000   
4  173871_4  173871      O_SENT  role_     10909  2011-10-05 03:40:47.8250000   

                   trace_start  timelapsed next_activity    milestone  \
0  2011-10-03 19:15:53.5370000           0     O_CREATED   O_SELECTED   
1  2011-10-03 18:35:59.2790000           0     O_CREATED   O_SELECTED   
2  2011-10-05 03:40:46.5260000           0    O_DECLINED  O_SENT_BACK   
3  2011-10-05 03:40:46.5260000           0    O_DECLINED  O_SENT_BACK   
4  2011-10-05 03:40:46.5260000           0    O_DECLINED  O_SENT_BACK   

   milestone_id  task_index  prefix_length  ac_index  rl_index  ne_index  
0             1           1              1         3        31         2  
1             1           1              1         3        45         2  
2             4           1              4         3        16         3  
3             4           2              4         2        16         3  
4             4           3              4         4        16         3

### Experiments

#### Load the Required Libraries

In [9]:
import os
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import callbacks, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support## Replicate Experiments
from pyflowchart import *
import plotly.express as px

from models.shared import *
from models.specialised import *
from models.explain import *

## Replicate Experiments

In [1]:
shared_acc = []
shared_prec = []
shared_rec = []
shared_f1 = []
runtime_shared = []
predict_shared = []

specialised_acc = []
specialised_prec = []
specialised_rec = []
specialised_f1 = []
runtime_specialised = []
predict_specialised = []

# Run the shared and specialised models 5 times each, to in the end compute the average performance scores
# for checking consistency
for i in list(range(5)):
    
    #### PRE-PROCESSING FOR EXPERIMENT
    
    # Randomize the data for checking consistency
    # To replicate without shuffling, comment the line below and change 'randomized_data' 'to log_df' in the succeeding line
    randomized_data = shuffle(log_df, random_state=i)

    # Split validation datasets
    log_df_train, log_df_test = split_train_test(randomized_data, 0.3) # 70%/30%
    
    # Normalize Numerical Features (timelapsed)
    numerical_features = ['timelapsed']
    log_df_train = normalize_events(log_df_train,args,numerical_features)
    log_df_test = normalize_events(log_df_test,args,numerical_features)
    
    # Reformat events: converting the dataframe into a dictionary to aid vectorization
    log_train = reformat_events(log_df_train, ac_index, rl_index,ne_index)
    log_test = reformat_events(log_df_test, ac_index, rl_index,ne_index)
    
    ### Vectorization: Convert the data dictionaries into trainable multi dimensional tensors
    
    # Obtain the maximum trc_len and cases for each set
    trc_len_train, cases_train = lengths(log_train)
    trc_len_test, cases_test = lengths(log_test)
    
    trc_len = max([trc_len_train, trc_len_test])
    
    
    # Converting the logs (dictionary) to a 3D-Tensor
    vec_train = vectorization(log_train,ac_index, rl_index, ne_index,trc_len,cases_train)
    vec_test = vectorization(log_test,ac_index, rl_index, ne_index,trc_len,cases_test)
    
    
    # Generating Initial Embedding Weights for the models
    ac_weights = to_categorical(sorted(index_ac.keys()), num_classes = len(ac_index))
    ac_weights[0] = 0 # embedding weights for label none = 0
    
    rl_weights =  to_categorical(sorted(index_rl.keys()), num_classes = len(rl_index))
    rl_weights[0] = 0 # embeddig weights for label none = 0
    
    
    # Converting the weights 
    weights = {'ac_weights':ac_weights, 'rl_weights':rl_weights, 'next_activity':len(ne_index)}
    
    # Converting the weights into a dictionary
    indexes = {'index_ac':index_ac, 'index_rl':index_rl,'index_ne':index_ne}
    
    # Converting the weights into a dictionary
    pre_index = {'ac_index':ac_index, 'rl_index':rl_index,'ne_index':ne_index}
    
    
    #### TRAINING AND TESTING SHARED MODEL
    
    print(f"Shared model {i+1}")
    
    start_shared = time.time()
    
    shared = shared_model(vec_train, weights, indexes, pre_index, args)
    shared.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    batch_size = 64 # 32, 64, 128, 256
    epochs = 200
    
    shared_history = shared_model_fit(vec_train, shared, indexes, pre_index, MY_WORKSPACE_DIR, batch_size, epochs, args)
    
    # Compute training time and append to list
    training_time_shared = time.time() - start_shared
    runtime_shared.append(training_time_shared)
    
    x_test,y_test = generate_inputs_shared(vec_test,args,indexes)
    
    accuracy = shared.evaluate(x_test, y_test, batch_size=batch_size)[1]
    
    # Append accuracy to list
    shared_acc.append(accuracy)
    
    # Compute time of making prediction given trained model
    start_shared_pred = time.time()
    y_pred_shared = shared.predict(x_test)
    predict_time_shared = (time.time() - start_shared_pred) / len(x_test)
    predict_shared.append(predict_time_shared)
    
    prec_rec_f1 = precision_recall_fscore_support(y_test.argmax(axis=1), y_pred_shared.argmax(axis=1), average='weighted')[0:3]
    precision, recall, f1_score = prec_rec_f1
    
    # Append precision, recall, and f1 scores to lists
    shared_prec.append(precision)
    shared_rec.append(recall)
    shared_f1.append(f1_score)
    
    # trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'shared_model_'+str(i)+subset+'.h5')
    # shared.save(trained_model)
    
    
    # Specialised model
    print("")
    print(f"Specialised model {i+1}")
    
    start_specialised = time.time()
    
    specialised = specialised_model(vec_train, weights, indexes, pre_index, args)
    specialised.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    batch_size = 64 # 32, 64, 128, 256
    epochs = 200
    
    specialised_history = specialised_model_fit(vec_train, specialised, indexes, pre_index, MY_WORKSPACE_DIR, batch_size, epochs, args)
    
    # Compute training time and append to list
    training_time_specialised = time.time() - start_specialised
    runtime_specialised.append(training_time_specialised)
    
    x_test,y_test = generate_inputs(vec_test,args,indexes, experiment)
    
    accuracy = specialised.evaluate(x_test, y_test, batch_size=batch_size)[1]
    
    # Append accuracy to list
    specialised_acc.append(accuracy)
    
    # Compute time of making prediction given trained model
    start_specialised_pred = time.time()
    y_pred_specialised = specialised.predict(x_test)
    predict_time_specialised = (time.time() - start_specialised_pred) / len(x_test)
    predict_specialised.append(predict_time_specialised)
    
    prec_rec_f1 = precision_recall_fscore_support(y_test.argmax(axis=1), y_pred_specialised.argmax(axis=1), average='weighted')[0:3]
    precision, recall, f1_score = prec_rec_f1
    
    # Append precision, recall, and f1 scores to lists
    specialised_prec.append(precision)
    specialised_rec.append(recall)
    specialised_f1.append(f1_score)
    
    # trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'specialised_model_'+str(i)+subset+'.h5')
    # specialised.save(trained_model)

### Get table of performance metrics

#### No Shuffling

In [22]:
shared_accuracy = np.mean(shared_acc)
shared_precision = np.mean(shared_prec)
shared_recall = np.mean(shared_rec)
shared_f1_score = np.mean(shared_f1)
runtime_shared_train = np.mean(runtime_shared)
runtime_predict_shared = np.mean(predict_shared)

specialised_accuracy = np.mean(specialised_acc)
specialised_precision = np.mean(specialised_prec)
specialised_recall = np.mean(specialised_rec)
specialised_f1_score = np.mean(specialised_f1)
runtime_specialised_train = np.mean(runtime_specialised)
runtime_predict_specialised = np.mean(predict_specialised)

accuracy = [shared_accuracy, specialised_accuracy]
precision = [shared_precision, specialised_precision]
recall = [shared_recall, specialised_recall]
f1 = [shared_f1_score, specialised_f1_score]
train_time = [runtime_shared_train, runtime_specialised_train]
pred_time = [runtime_predict_shared, runtime_predict_specialised]

In [23]:
std_shared_accuracy = round(np.std(shared_acc), 4)
std_shared_precision = round(np.std(shared_prec), 4)
std_shared_recall = round(np.std(shared_rec), 4)
std_shared_f1_score = round(np.std(shared_f1), 4)
std_runtime_shared_train = round(np.std(runtime_shared), 4)
std_runtime_predict_shared = round(np.std(predict_shared), 4)


std_specialised_accuracy = round(np.std(specialised_acc), 4)
std_specialised_precision = round(np.std(specialised_prec), 4)
std_specialised_recall = round(np.std(specialised_rec), 4)
std_specialised_f1_score = round(np.std(specialised_f1), 4)
std_runtime_specialised_train = round(np.std(runtime_specialised), 4)
std_runtime_predict_specialised = round(np.std(predict_specialised), 4)

std_accuracy = [std_shared_accuracy, std_specialised_accuracy]
std_precision = [std_shared_precision, std_specialised_precision]
std_recall = [std_shared_recall, std_specialised_recall]
std_f1 = [std_shared_f1_score, std_specialised_f1_score]
std_train_time = [std_runtime_shared_train, std_runtime_specialised_train]
std_pred_time = [std_runtime_predict_shared, std_runtime_predict_specialised]

In [25]:
performance_metrics = pd.DataFrame({"Model":["Shared", "Specialised"], "Event log":["BPIC 2012 O", "BPIC 2012 O"], "Accuracy": accuracy, "std Accuracy":std_accuracy, "Precision": precision, "std Precision": std_precision, "Recall": recall, "std Recall": std_recall, "F1-Score": f1, "std F1-Score": std_f1, "Training Time": train_time, "std Training Time": std_train_time, "Prediction Time": pred_time, "std Prediction Time": std_pred_time})
performance_metrics

Model    Event log  Accuracy  std Accuracy  Precision  std Precision  \
0       Shared  BPIC 2012 O  0.820241        0.0007   0.815118          0.001   
1  Specialised  BPIC 2012 O  0.814248        0.0038   0.812907          0.010   

     Recall  std Recall  F1-Score  std F1-Score  Training Time  \
0  0.820241      0.0007  0.804208        0.0006     378.599264   
1  0.814248      0.0038  0.798096        0.0009    1141.270693   

   std Training Time  Prediction Time  std Prediction Time  
0           174.8364         3.895790               1.1243  
1           212.3929         7.050613               0.2276

#### Shuffling

In [18]:
shared_accuracy = np.mean(shared_acc)
shared_precision = np.mean(shared_prec)
shared_recall = np.mean(shared_rec)
shared_f1_score = np.mean(shared_f1)
runtime_shared_train = np.mean(runtime_shared)
runtime_predict_shared = np.mean(predict_shared)

specialised_accuracy = np.mean(specialised_acc)
specialised_precision = np.mean(specialised_prec)
specialised_recall = np.mean(specialised_rec)
specialised_f1_score = np.mean(specialised_f1)
runtime_specialised_train = np.mean(runtime_specialised)
runtime_predict_specialised = np.mean(predict_specialised)

accuracy = [shared_accuracy, specialised_accuracy]
precision = [shared_precision, specialised_precision]
recall = [shared_recall, specialised_recall]
f1 = [shared_f1_score, specialised_f1_score]
train_time = [runtime_shared_train, runtime_specialised_train]
pred_time = [runtime_predict_shared, runtime_predict_specialised]

In [19]:
std_shared_accuracy = round(np.std(shared_acc), 4)
std_shared_precision = round(np.std(shared_prec), 4)
std_shared_recall = round(np.std(shared_rec), 4)
std_shared_f1_score = round(np.std(shared_f1), 4)
std_runtime_shared_train = round(np.std(runtime_shared), 4)
std_runtime_predict_shared = round(np.std(predict_shared), 4)


std_specialised_accuracy = round(np.std(specialised_acc), 4)
std_specialised_precision = round(np.std(specialised_prec), 4)
std_specialised_recall = round(np.std(specialised_rec), 4)
std_specialised_f1_score = round(np.std(specialised_f1), 4)
std_runtime_specialised_train = round(np.std(runtime_specialised), 4)
std_runtime_predict_specialised = round(np.std(predict_specialised), 4)

std_accuracy = [std_shared_accuracy, std_specialised_accuracy]
std_precision = [std_shared_precision, std_specialised_precision]
std_recall = [std_shared_recall, std_specialised_recall]
std_f1 = [std_shared_f1_score, std_specialised_f1_score]
std_train_time = [std_runtime_shared_train, std_runtime_specialised_train]
std_pred_time = [std_runtime_predict_shared, std_runtime_predict_specialised]

In [20]:
performance_metrics = pd.DataFrame({"Model":["Shared", "Specialised"], "Event log":["BPIC 2012 O", "BPIC 2012 O"], "Accuracy": accuracy, "std Accuracy":std_accuracy, "Precision": precision, "std Precision": std_precision, "Recall": recall, "std Recall": std_recall, "F1-Score": f1, "std F1-Score": std_f1, "Training Time": train_time, "std Training Time": std_train_time, "Prediction Time": pred_time, "std Prediction Time": std_pred_time})
performance_metrics

Model    Event log  Accuracy  std Accuracy  Precision  std Precision  \
0       Shared  BPIC 2012 A  0.780286        0.0047   0.776863         0.0113   
1  Specialised  BPIC 2012 A  0.778160        0.0075   0.782748         0.0186   

     Recall  std Recall  F1-Score  std F1-Score  Training Time  \
0  0.780286      0.0047  0.763833        0.0022     317.424283   
1  0.778160      0.0075  0.761790        0.0045    1243.111465   

   std Training Time  Prediction Time  std Prediction Time  
0            28.0115         7.737332               6.8748  
1           252.0187         6.554780               2.0231

#### Load Saved Model and Evaluate

In [ ]:
# shared_trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'shared_model_'+subset+'.h5')
# shared = load_model(shared_trained_model)

#### Load Saved Model and Evaluate

In [ ]:
# trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'specialised_model_'+subset+'.h5')
# specialised = load_model(trained_model)